In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels as sm
import lightgbm as lgb
from prophet import Prophet
import datetime
import pmdarima
import tensorflow as tf

import warnings
warnings.filterwarnings('ignore')

/Users/gokberk/miniconda3/envs/ml4hc_project2/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


In [4]:
# define performance metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

def rmse(y_true, y_pred):
    return mean_squared_error(y_true,y_pred,squared=False)

def mae(y_true, y_pred):
    return mean_absolute_error(y_true, y_pred)

def mape(y_true, y_pred):
    return mean_absolute_percentage_error(y_true, y_pred)

def smape(y_true, y_pred):
    return (200/len(y_true))*np.sum(np.abs(y_true - y_pred)/(np.abs(y_true) + np.abs(y_pred)))

In [5]:
df = pd.read_excel('data/M3C.xls','M3Year')

In [6]:
df.head()

,Series,N,NF,Category,Starting Year,Unnamed: 5,1,2,3,4,...,38,39,40,41,42,43,44,45,46,47
0,N 1,20,6,MICRO,1975,1,940.66,1084.86,1244.98,1445.02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,N 2,20,6,MICRO,1975,1,1991.05,2306.40,2604.00,2992.30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,N 3,20,6,MICRO,1975,1,1461.57,1692.50,2193.82,2459.68,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,N 4,20,6,MICRO,1975,1,744.54,1105.16,1417.40,1838.04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,N 5,20,6,MICRO,1975,1,4977.18,5248.00,5370.00,6184.89,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## naive

In [6]:
# read and process data model-agnostic
# train and evaluate model model-specific

valid_mae, valid_rmse, valid_mape, valid_smape = 0., 0., 0., 0.
test_mae, test_rmse, test_mape, test_smape = 0., 0., 0., 0.

for index, row in df.iterrows():
    series_len = row['N']
    forecast_H = row['NF']

    train_df = row.loc[1:series_len-2*forecast_H]
    valid_df = row.loc[series_len-2*forecast_H+1:series_len-forecast_H]
    test_df = row.loc[series_len-forecast_H+1:series_len]

    # fit
    y_pred_valid = np.repeat(train_df.iloc[-1],len(valid_df))
    y_true_valid = valid_df.values

    y_pred_test = np.repeat(valid_df.iloc[-1],len(test_df))
    y_true_test = test_df.values

    # evaluate
    valid_mae += mae(y_true_valid, y_pred_valid)
    valid_rmse += rmse(y_true_valid, y_pred_valid)
    valid_mape += mape(y_true_valid, y_pred_valid)
    valid_smape += smape(y_true_valid, y_pred_valid)
    
    test_mae += mae(y_true_test, y_pred_test)
    test_rmse += rmse(y_true_test, y_pred_test)
    test_mape += mape(y_true_test, y_pred_test)
    test_smape += smape(y_true_test, y_pred_test)

    #print(mae(y_true_test, y_pred_test), rmse(y_true_test, y_pred_test), mape(y_true_test, y_pred_test))


valid_mae /= len(df)
valid_rmse /= len(df)
valid_mape /= len(df)
valid_smape /= len(df)

test_mae /= len(df)
test_rmse /= len(df)
test_mape /= len(df)
test_smape /= len(df)


print(f'valid | MAE: {round(valid_mae,2)}, RMSE: {round(valid_rmse,2)}, MAPE: {round(valid_mape,2)}, SMAPE: {round(valid_smape,2)}')
print(f'test  | MAE: {round(test_mae,2)}, RMSE: {round(test_rmse,2)}, MAPE: {round(test_mape,2)}, SMAPE: {round(test_smape,2)}')

valid | MAE: 1037.7, RMSE: 1203.71, MAPE: 0.23, SMAPE: 21.45
test  | MAE: 1025.84, RMSE: 1178.59, MAPE: 0.21, SMAPE: 17.88


## mean

In [7]:
# read and process data model-agnostic
# train and evaluate model model-specific

valid_mae, valid_rmse, valid_mape = 0., 0., 0.
test_mae, test_rmse, test_mape = 0., 0., 0.

for index, row in df.iterrows():
    series_len = row['N']
    forecast_H = row['NF']

    train_df = row.loc[1:series_len-2*forecast_H]
    valid_df = row.loc[series_len-2*forecast_H+1:series_len-forecast_H]
    test_df = row.loc[series_len-forecast_H+1:series_len]

    # fit
    y_pred_valid = np.repeat(np.mean(train_df),len(valid_df))
    y_true_valid = valid_df.values

    y_pred_test = np.repeat(np.mean(pd.concat([train_df, valid_df])),len(test_df))
    y_true_test = test_df.values

    # evaluate
    valid_mae += mae(y_true_valid, y_pred_valid)
    valid_rmse += rmse(y_true_valid, y_pred_valid)
    valid_mape += mape(y_true_valid, y_pred_valid)
    valid_smape += smape(y_true_valid, y_pred_valid)
    
    test_mae += mae(y_true_test, y_pred_test)
    test_rmse += rmse(y_true_test, y_pred_test)
    test_mape += mape(y_true_test, y_pred_test)
    test_smape += smape(y_true_test, y_pred_test)

    #print(mae(y_true_test, y_pred_test), rmse(y_true_test, y_pred_test), mape(y_true_test, y_pred_test))


valid_mae /= len(df)
valid_rmse /= len(df)
valid_mape /= len(df)
valid_smape /= len(df)

test_mae /= len(df)
test_rmse /= len(df)
test_mape /= len(df)
test_smape /= len(df)


print(f'valid | MAE: {round(valid_mae,2)}, RMSE: {round(valid_rmse,2)}, MAPE: {round(valid_mape,2)}, SMAPE: {round(valid_smape,2)}')
print(f'test  | MAE: {round(test_mae,2)}, RMSE: {round(test_rmse,2)}, MAPE: {round(test_mape,2)}, SMAPE: {round(test_smape,2)}')

valid | MAE: 1814.99, RMSE: 1946.95, MAPE: 0.39, SMAPE: 42.1
test  | MAE: 2294.0, RMSE: 2403.32, MAPE: 0.4, SMAPE: 43.65


## 1-step naive (not applicable)

In [8]:
# read and process data model-agnostic
# train and evaluate model model-specific

valid_mae, valid_rmse, valid_mape = 0., 0., 0.
test_mae, test_rmse, test_mape = 0., 0., 0.

for index, row in df.iterrows():
    series_len = row['N']
    forecast_H = row['NF']

    train_df = row.loc[1:series_len-2*forecast_H]
    valid_df = row.loc[series_len-2*forecast_H+1:series_len-forecast_H]
    test_df = row.loc[series_len-forecast_H+1:series_len]

    # fit
    y_pred_valid = valid_df.shift(1).values
    y_pred_valid[0] = train_df.iloc[-1]

    y_true_valid = valid_df.values

    y_pred_test = test_df.shift(1).values
    y_pred_test[0] = valid_df.iloc[-1]

    y_true_test = test_df.values

    # evaluate
    valid_mae += mae(y_true_valid, y_pred_valid)
    valid_rmse += rmse(y_true_valid, y_pred_valid)
    valid_mape += mape(y_true_valid, y_pred_valid)
    valid_smape += smape(y_true_valid, y_pred_valid)
    
    test_mae += mae(y_true_test, y_pred_test)
    test_rmse += rmse(y_true_test, y_pred_test)
    test_mape += mape(y_true_test, y_pred_test)
    test_smape += smape(y_true_test, y_pred_test)

    #print(mae(y_true_test, y_pred_test), rmse(y_true_test, y_pred_test), mape(y_true_test, y_pred_test))


valid_mae /= len(df)
valid_rmse /= len(df)
valid_mape /= len(df)
valid_smape /= len(df)

test_mae /= len(df)
test_rmse /= len(df)
test_mape /= len(df)
test_smape /= len(df)


print(f'valid | MAE: {round(valid_mae,2)}, RMSE: {round(valid_rmse,2)}, MAPE: {round(valid_mape,2)}, SMAPE: {round(valid_smape,2)}')
print(f'test  | MAE: {round(test_mae,2)}, RMSE: {round(test_rmse,2)}, MAPE: {round(test_mape,2)}, SMAPE: {round(test_smape,2)}')

valid | MAE: 507.31, RMSE: 618.25, MAPE: 0.12, SMAPE: 10.5
test  | MAE: 526.42, RMSE: 641.03, MAPE: 0.12, SMAPE: 9.5


## prophet

In [ ]:
valid_mae, valid_rmse, valid_mape, valid_smape = 0., 0., 0., 0.
test_mae, test_rmse, test_mape, test_smape = 0., 0., 0., 0.

for index, row in df.iterrows():
    series_len = row['N']
    forecast_H = row['NF']

    y = row.loc[1:series_len]
    
    # change the added value for each frequency
    base_date = datetime.datetime.strptime(str(row['Starting Year']), '%Y')
    dates=[]
    for i in range(len(y)):
        dates.append(datetime.datetime.strptime(str(base_date.year + i), '%Y'))

    whole_df = pd.DataFrame.from_dict({'ds':dates,'y':y})

    train_df = whole_df.iloc[:-2*forecast_H]
    valid_df = whole_df.iloc[-2*forecast_H:-forecast_H]
    test_df = whole_df.iloc[-forecast_H:]
    
    # fit & cross validation
    model = Prophet(yearly_seasonality=False, weekly_seasonality=False, daily_seasonality=False)

    model.fit(train_df)

    future = model.make_future_dataframe(periods=len(valid_df))

    forecast = model.predict(future)

    y_pred_valid = forecast['yhat'].iloc[-len(valid_df):]
    y_true_valid = valid_df['y']

    # fit on selected hypers
    model = Prophet(yearly_seasonality=False, weekly_seasonality=False, daily_seasonality=False)

    model.fit(pd.concat([train_df,valid_df]))

    future = model.make_future_dataframe(periods=len(test_df))

    forecast = model.predict(future)

    y_pred_test = forecast['yhat'].iloc[-len(test_df):]
    y_true_test = test_df['y']

    # evaluate
    valid_mae += mae(y_true_valid, y_pred_valid)
    valid_rmse += rmse(y_true_valid, y_pred_valid)
    valid_mape += mape(y_true_valid, y_pred_valid)
    valid_smape += smape(y_true_valid, y_pred_valid)
    
    test_mae += mae(y_true_test, y_pred_test)
    test_rmse += rmse(y_true_test, y_pred_test)
    test_mape += mape(y_true_test, y_pred_test)
    test_smape += smape(y_true_test, y_pred_test)



valid_mae /= len(df)
valid_rmse /= len(df)
valid_mape /= len(df)
valid_smape /= len(df)

test_mae /= len(df)
test_rmse /= len(df)
test_mape /= len(df)
test_smape /= len(df)


print(f'valid | MAE: {round(valid_mae,2)}, RMSE: {round(valid_rmse,2)}, MAPE: {round(valid_mape,2)}, SMAPE: {round(valid_smape,2)}')
print(f'test  | MAE: {round(test_mae,2)}, RMSE: {round(test_rmse,2)}, MAPE: {round(test_mape,2)}, SMAPE: {round(test_smape,2)}')

## arima

In [ ]:
test_mae, test_rmse, test_mape, test_smape = 0., 0., 0., 0.

for index, row in df.iterrows():

    series_len = row['N']
    forecast_H = row['NF']
    
    train_df = row.loc[1:series_len-2*forecast_H]
    valid_df = row.loc[series_len-2*forecast_H+1:series_len-forecast_H]
    test_df = row.loc[series_len-forecast_H+1:series_len]
    
    # fit
    model = pmdarima.AutoARIMA(seasonal=True)
    
    model.fit(pd.concat([train_df, valid_df]))

    y_pred_test = model.predict(n_periods=forecast_H)
    y_true_test = test_df.values

    # evaluate
    test_mae += mae(y_true_test, y_pred_test)
    test_rmse += rmse(y_true_test, y_pred_test)
    test_mape += mape(y_true_test, y_pred_test)
    test_smape += smape(y_true_test, y_pred_test)

test_mae /= len(df)
test_rmse /= len(df)
test_mape /= len(df)
test_smape /= len(df)

print(f'test  | MAE: {round(test_mae,2)}, RMSE: {round(test_rmse,2)}, MAPE: {round(test_mape,2)}, SMAPE: {round(test_smape,2)}')

## TCN

In [ ]:
def process_series(df, input_size, output_size, stride):

    X = np.empty(shape=(1,input_size,1))
    y = np.empty(shape=(1,input_size))

    for idx, _ in df.iterrows():
        
        if idx % stride == 0 and idx+input_size+output_size <= len(df):
            
            input_values = (df.iloc[idx:idx+input_size]).to_numpy()
            input_values = np.expand_dims(input_values,0)

            output_values = (df.iloc[idx+output_size:idx+input_size+output_size]).to_numpy().T
            
            X = np.concatenate((X,input_values))
            y = np.concatenate((y,output_values))
    
    return X[1:],y[1:]

In [ ]:
# hyperparameter tuning for tcn model
from sklearn.preprocessing import StandardScaler

forecast_H = df.loc[0,'NF']
lookback_H = df.loc[0,'NF'] * 2  # hyperparameter

all_X_train = np.empty(shape=(1,lookback_H,1))
all_y_train = np.empty(shape=(1,lookback_H))

prediction_dict={}

# iterate over all TS and extract seqns eligible for training
for index, row in df.iterrows():

    series_len = row['N']
    
    train_df = row.loc[1:series_len-forecast_H]
    test_df = row.loc[series_len-forecast_H+1-lookback_H:series_len]

    train_df = np.expand_dims(train_df.to_numpy(),axis=1)
    test_df = np.expand_dims(test_df.to_numpy(),axis=1)

    scaler = StandardScaler()
    train_df = pd.DataFrame(np.squeeze(scaler.fit_transform(train_df)))
    test_df = pd.DataFrame(np.squeeze(scaler.transform(test_df)))

    train_X, train_y = process_series(train_df, lookback_H, forecast_H, 1)
    test_X, test_y = process_series(test_df, lookback_H, forecast_H, 1)

    all_X_train = np.concatenate((all_X_train,train_X))
    all_y_train = np.concatenate((all_y_train,train_y))

    prediction_dict[row['Series']] = {'scaler':scaler, 'test_X':test_X, 'test_y':test_y}

all_X_train = all_X_train[1:]
all_y_train = all_y_train[1:]

In [ ]:
import tensorflow as tf
import tensorflow.keras.layers as tfkl

from model import TCN

model = TCN(num_layers=3,num_filters=20,kernel_size=3,dilation_base=2)

loss = tf.keras.losses.MeanSquaredError()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)

model.compile(loss=loss, optimizer=optimizer)

model.fit(x=all_X_train, y=all_y_train, epochs=20, batch_size=64)

In [ ]:
test_mae, test_rmse, test_mape, test_smape = 0., 0., 0., 0.

for key in prediction_dict.keys():
    elem = prediction_dict[key]
    scaler, test_X, test_y = elem['scaler'], elem['test_X'], elem['test_y']
    
    test_preds = np.expand_dims((model.predict(test_X)[:,-forecast_H:,0]).flatten(),0)
    y_pred_test = np.squeeze(scaler.inverse_transform(test_preds))

    y_true_test = np.squeeze(scaler.inverse_transform(test_y[:,-forecast_H:]))

    test_mae += mae(y_true_test, y_pred_test)
    test_rmse += rmse(y_true_test, y_pred_test)
    test_mape += mape(y_true_test, y_pred_test)
    test_smape += smape(y_true_test, y_pred_test)

test_mae /= len(df)
test_rmse /= len(df)
test_mape /= len(df)
test_smape /= len(df)

print(f'test  | MAE: {round(test_mae,2)}, RMSE: {round(test_rmse,2)}, MAPE: {round(test_mape,2)}, SMAPE: {round(test_smape,2)}')

## n-beats

In [8]:
def process_series(df, input_size, output_size, stride):

    X = np.empty(shape=(1,input_size))
    y = np.empty(shape=(1,output_size))

    for idx, _ in df.iterrows():
        
        if idx % stride == 0 and idx+input_size+output_size <= len(df):
            
            input_values = (df.iloc[idx:idx+input_size]).to_numpy().T

            output_values = (df.iloc[idx+input_size:idx+input_size+output_size]).to_numpy().T
            
            X = np.concatenate((X,input_values))
            y = np.concatenate((y,output_values))
    
    return X[1:],y[1:]

In [9]:
from sklearn.preprocessing import StandardScaler

forecast_H = 2 #df.loc[0,'NF']
lookback_H = forecast_H*7 #df.loc[0,'NF'] * 2  # hyperparameter

all_X_train = np.empty(shape=(1,lookback_H))
all_y_train = np.empty(shape=(1,forecast_H))

prediction_dict={}

# iterate over all TS and extract seqns eligible for training
for index, row in df.iterrows():

    series_len = row['N']
    
    train_df = row.loc[1:series_len-forecast_H]
    test_df = row.loc[series_len-forecast_H+1-lookback_H:series_len]

    train_df = np.expand_dims(train_df.to_numpy(),axis=1)
    test_df = np.expand_dims(test_df.to_numpy(),axis=1)

    scaler = StandardScaler()
    train_df = pd.DataFrame(np.squeeze(scaler.fit_transform(train_df)))
    test_df = pd.DataFrame(np.squeeze(scaler.transform(test_df)))

    train_X, train_y = process_series(train_df, lookback_H, forecast_H, 1)
    test_X, test_y = process_series(test_df, lookback_H, forecast_H, 1)

    all_X_train = np.concatenate((all_X_train,train_X))
    all_y_train = np.concatenate((all_y_train,train_y))

    prediction_dict[row['Series']] = {'scaler':scaler, 'test_X':test_X, 'test_y':test_y}

all_X_train = all_X_train[1:]
all_y_train = all_y_train[1:]

In [10]:
from model import NBEATS

model = NBEATS(stacks=5, blocks=5, width=64, forecast_H=forecast_H, lookback_H=lookback_H)

loss = tf.keras.losses.MeanSquaredError()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)

early_stopping = tf.keras.callbacks.EarlyStopping(patience=5,verbose=1)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(factor=0.2,patience=2,cooldown=3,verbose=1)

model.compile(loss=loss, optimizer=optimizer)

model.fit(x=all_X_train, y=all_y_train, epochs=10, batch_size=64)

2022-12-01 17:07:05.533640: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/10
115/115 [==============================] - 7s 12ms/step - loss: 0.4784
Epoch 2/10
115/115 [==============================] - 1s 12ms/step - loss: 0.3263
Epoch 3/10
115/115 [==============================] - 1s 12ms/step - loss: 0.2932
Epoch 4/10
115/115 [==============================] - 1s 12ms/step - loss: 0.2637
Epoch 5/10
115/115 [==============================] - 1s 12ms/step - loss: 0.2434
Epoch 6/10
115/115 [==============================] - 1s 12ms/step - loss: 0.2262
Epoch 7/10
115/115 [==============================] - 1s 12ms/step - loss: 0.2104
Epoch 8/10
115/115 [==============================] - 1s 12ms/step - loss: 0.1995
Epoch 9/10
115/115 [==============================] - 1s 12ms/step - loss: 0.1849
Epoch 10/10
115/115 [==============================] - 1s 12ms/step - loss: 0.1759


In [11]:
test_mae, test_rmse, test_mape, test_smape = 0., 0., 0., 0.

for key in prediction_dict.keys():
    elem = prediction_dict[key]
    scaler, test_X, test_y = elem['scaler'], elem['test_X'], elem['test_y']
    
    test_preds = np.expand_dims(model.predict(test_X).flatten(),0)
    y_pred_test = np.squeeze(scaler.inverse_transform(test_preds))

    y_true_test = np.squeeze(scaler.inverse_transform(test_y))

    test_mae += mae(y_true_test, y_pred_test)
    test_rmse += rmse(y_true_test, y_pred_test)
    test_mape += mape(y_true_test, y_pred_test)
    test_smape += smape(y_true_test, y_pred_test)

test_mae /= len(df)
test_rmse /= len(df)
test_mape /= len(df)
test_smape /= len(df)

print(f'test  | MAE: {round(test_mae,2)}, RMSE: {round(test_rmse,2)}, MAPE: {round(test_mape,2)}, SMAPE: {round(test_smape,2)}')

test  | MAE: 879.48, RMSE: 948.51, MAPE: 0.16, SMAPE: 14.58


### yearly
### + nbeats
forecast (H): 6, lookback: 2H -> SMAPE: 25.5
forecast (H): 5, lookback: 2H -> SMAPE: 25.48
forecast (H): 5, lookback: 3H -> SMAPE: 24.3
forecast (H): 3, lookback: 2H -> SMAPE: 19.47
forecast (H): 3, lookback: 4H -> SMAPE: 17.76
forecast (H): 2, lookback: 7H -> SMAPE: 14.71
forecast (H): 1, lookback: 2H -> SMAPE: 11.51
forecast (H): 1, lookback: 4H -> SMAPE: 11.98
forecast (H): 1, lookback: 7H -> SMAPE: 13.17